In [1]:
from pynq import Overlay
from pynq import MMIO
from pynq.lib.iic import AxiIIC
import cffi, time
import threading
import csv

overlay = Overlay("/home/xilinx/pynq/overlays/axi_test.bit")

In [2]:
ADDR_BASE = 0x41600000
ADDR_RANGE = 0x10000
i2c_obj = MMIO(ADDR_BASE,ADDR_RANGE)


In [3]:
i2c_obj.write(292,0)

In [4]:
iic_ip = overlay.ip_dict['axi_iic_0']
iic = AxiIIC(iic_ip)

ffi = cffi.FFI()

In [5]:
def read_(addr,reg,length=1):
    iic.send(addr,bytes([reg]),1,0)
    buf = ffi.new("unsigned char[]",length)
    iic.receive(addr, buf, length, 0)
    return [buf[i] for i in range(length)]

def write_(addr, reg, value):
    iic.send(addr, bytes([reg,value]), 2, 0)

def to_int16(low,high):
    val = (high << 8) | low
    if val & 0x8000:
        val -= 1 << 16
    return val
print(overlay.ip_dict.keys())

dict_keys(['axi_iic_0', 'processing_system7_0'])


In [6]:
# BN0055 connection check
BN0055_addr = 0x28
chip_addr = 0x00
chip_id = read_(BN0055_addr, 0x00, 1)[0]
print("CHIP ID:", hex(chip_id)) #0xA0

CHIP ID: 0xa0


In [7]:
#NDOF mode on 0x0C

write_(BN0055_addr, 0x3D, 0X0C)
time.sleep(0.1)

In [8]:
#calibration check
cal = read_(BN0055_addr, 0x35, 1)[0]
sys = (cal >> 6) & 0x03
gyro = (cal >> 4) & 0x03
accel = (cal >> 2) & 0x03
mag = cal & 0x03

print(f"Cal : sys = {sys} Gyro = {gyro} accel = {accel} mag = {mag}")

Cal : sys = 0 Gyro = 3 accel = 3 mag = 3


In [9]:
# roboclaw encoder 
import serial.tools.list_ports

ports = serial.tools.list_ports.comports()

for port in ports:
    print(f"Device: {port.device} | Description: {port.description}")

import sys
sys.path.append("/home/xilinx/pynq/roboclaw")  # 실제 roboclaw_3.py가 있는 경로
from roboclaw_3 import Roboclaw

rc = Roboclaw("/dev/ttyACM0", 38400)  # 또는 ttyUSB0
rc.Open()

rc_address = 0x80

# 패턴 예시: (왼쪽속도, 오른쪽속도)
pattern = [(10,10), (10,15), (10,16), (10,14),(10,15),(10,16)]
pattern_0 = [(0,0), (0,0), (0,0), (0,0)]

durations = [2, 1, 2, 1, 1, 1]  # 각 패턴 지속 시간
pattern_idx = 0
start_time_motor = time.time()

Device: /dev/ttyACM0 | Description: USB Roboclaw 2x30A


In [10]:
# ======================
# CSV 준비
# ======================
csv_file = "rover_data.csv"
with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([
        "time","label", "delta_encL", "delta_encR",
        "heading", "cmdL", "cmdR"
    ])


In [11]:
def main_loop():
    # 전역 변수 선언: 모터 제어 및 센서 로깅에 필요한 모든 변수를 선언합니다.
    global pattern_idx, start_time_motor, cmdL, cmdR
    global prev_encL, prev_encR
    
    prev_encL, prev_encR = rc.ReadEncM1(rc_address)[1], rc.ReadEncM2(rc_address)[1]
    
    data = read_(BN0055_addr, 0x08, 44)
    initial_heading = to_int16(data[18], data[19]) / 16.0
    print(initial_heading)
    
    cmdL, cmdR = 10, 10
    
    
    # 목표 주기 설정 (20Hz = 0.05초)
    LOOP_PERIOD = 0.05 
    CORRECTION_PERIOD = 0.15
    
    last_correction_time = time.time()
    
    while True:
        loop_start_time = time.time()
        

        
        curr_time = loop_start_time
        
        # 엔코더 읽기 및 delta 계산
        encL = rc.ReadEncM1(rc_address)[1]
        encR = rc.ReadEncM2(rc_address)[1]
        
        # delta 계산 (이전 값 사용)
        delta_encL = encL - prev_encL
        delta_encR = encR - prev_encR
        
        # 다음 루프를 위한 이전 값 갱신
        prev_encL, prev_encR = encL, encR

        # IMU 읽기
        data = read_(BN0055_addr,0x08,44)
        heading = to_int16(data[18],data[19]) / 16.0
        
        

        
        label = "forward"
        
        if time.time() - last_correction_time >= CORRECTION_PERIOD:

            diff = heading - initial_heading

            if -0.25 <= diff <= 0.25:
                correction = 0

            elif diff < -1:
                correction = +3    # 오른쪽으로 많이 돌아감

            elif diff < -0.25:
                correction = +2    # 오른쪽으로 조금 돌아감

            elif diff > 1:
                correction = -3    # 왼쪽으로 많이 돌아감

            elif diff > 0.25:
                correction = -2    # 왼쪽으로 조금 돌아감

            
            cmdR += correction

            
            cmdR = max(0, min(127, cmdR))

            
            last_correction_time = time.time()
            
        
        rc.ForwardM1(rc_address, cmdL)
        rc.ForwardM2(rc_address, cmdR)

        # CSV 기록
        with open(csv_file, mode='a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                curr_time, label, delta_encL, delta_encR, heading, cmdL, cmdR
            ])
            
        # =======================# 3. 루프 시간 맞추기 (Loop Timing)# =======================
        loop_end_time = time.time()
        time_elapsed = loop_end_time - loop_start_time
        
        # 남은 시간만큼 대기하여 전체 루프 시간을 LOOP_PERIOD에 맞춥니다.
        sleep_time = LOOP_PERIOD - time_elapsed
        if sleep_time > 0:
            time.sleep(sleep_time)

In [12]:
main_loop()

325.875



KeyboardInterrupt



In [ ]:
ls /dev/*

In [ ]:
ports = serial.tools.list_ports.comports()

for port in ports:
    print(f"Device: {port.device} | Description: {port.description}")

In [18]:
rc.ForwardM1(rc_address, 0)
rc.ForwardM2(rc_address, 0)

True